<a href="https://colab.research.google.com/github/JValdez777/asl-translator-notebook/blob/main/aiASLProject_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras mediapipe-model-maker
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/

In [ ]:
import sys
sys.setrecursionlimit(100000) # Increase to a suitable value
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers
import os
import numpy as np
import json
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import Counter
from mediapipe.python.solutions.pose import Pose
from mediapipe.python.solutions.pose import POSE_CONNECTIONS
from mediapipe.python.solutions.drawing_utils import draw_landmarks
import cv2

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
# ============================
# Customizable Settings
# ============================
ASL_DIR = '/content/drive/MyDrive/ASL_Project/ASL_Alpha'
MODEL_DIR = '/content/drive/MyDrive/ASL_Project/ASL_Model2'
config_path = os.path.join(MODEL_DIR, 'model_config.json')
delete_previous_model = False  # Option to delete previous model

# Model parameters
set_num_epochs = 200  # Lower; better generalization with early stopping
set_batch_size = 16   # Smaller batches can help sequence models
set_patience = 20
set_patience_sensitivity = 0.0005
set_min_lr = 1e-6
set_factor = 0.5

# Architecture parameters
set_lstm_units = 128            # More capacity to learn from sequences
set_dense_units = 64            # Better compression of LSTM output
set_dropout_rate = 0.3          # Moderate regularization
set_bidirectional = True
set_learning_rate = 1e-4        # Keep this low for stability
set_num_lstm_layers = 2         # More temporal abstraction
set_num_dense_layers = 1
set_activation = 'relu'
set_regularizer_strength = 0.0001  # L2 regularization, reduced
set_pose_noise_std = 0.01  # Add a tiny bit of pose jittering

# Preprocessing parameters
target_size = (64, 64)
channels = 3

In [ ]:
# ============================
# Initialization & Setup
# ============================
# Handle model directory
os.makedirs(MODEL_DIR, exist_ok=True)
if delete_previous_model and os.path.exists(MODEL_DIR):
    shutil.rmtree(MODEL_DIR)
    os.makedirs(MODEL_DIR)

# Function to extract pose keypoints from an image
def extract_pose_keypoints(image, pose_model):
    results = pose_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.pose_landmarks:
        keypoints = []
        for lm in results.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])
        return keypoints
    else:
        # Return all zeros if no pose is detected
        return [0.0] * (33 * 4)

# ============================
# Calculate max_frames
# ============================
def calculate_max_frames(directory):
    max_frames = 0
    for sign in tqdm(os.listdir(directory), desc="Signs"):
        sign_dir = os.path.join(directory, sign)
        if os.path.isdir(sign_dir):
            for video in os.listdir(sign_dir):
                video_path = os.path.join(sign_dir, video)
                if os.path.isdir(video_path):
                    try:
                        frame_count = len([f for f in os.listdir(video_path) if f.endswith('.jpg')])
                        if frame_count > max_frames:
                            max_frames = frame_count
                    except Exception as e:
                        print(f"Error accessing {video_path}: {e}")
    return max_frames

max_frames = calculate_max_frames(ASL_DIR)
print(f"Calculated maximum frames across all videos: {max_frames}")

# ============================
# Data Augmentation for Pose Data
# ============================
def augment_pose_sequence(pose_data, noise_std=set_pose_noise_std):
    """Augment pose sequence data by adding noise and transformations."""
    augmented_pose = pose_data.copy()

    # Adding noise to pose data (random jitter)
    noise = np.random.normal(0, noise_std, augmented_pose.shape)
    augmented_pose += noise

    # Optionally, add more augmentations like scaling, flipping, or rotating here if needed

    return augmented_pose

def augment_video_pose(video_dir, augmentations_needed=1):
    """Create augmented copies of a video by applying random transformations."""
    pose_file = os.path.join(video_dir, 'processed_poses.npy')
    if not os.path.exists(pose_file):
        print(f"Pose data not found for {video_dir}")
        return []

    pose_data = np.load(pose_file)
    augmented_dirs = []

    parent_dir = os.path.dirname(video_dir)
    base_name = os.path.basename(video_dir)

    augmented_index = 0
    for _ in range(augmentations_needed):
        new_video_name = f'augmentedposes_{augmented_index}'
        new_video_path = os.path.join(parent_dir, new_video_name)
        os.makedirs(new_video_path, exist_ok=True)

        augmented_pose_data = augment_pose_sequence(pose_data)
        np.save(os.path.join(new_video_path, 'processed_poses.npy'), augmented_pose_data)

        augmented_dirs.append(new_video_path)
        augmented_index += 1

    return augmented_dirs

# ============================
# Preprocessing Generator for Pose Data
# ============================
class PoseDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, video_paths, labels, label_to_index, batch_size):
        self.video_paths = video_paths
        self.labels = labels
        self.label_to_index = label_to_index
        self.batch_size = batch_size
        self.indices = np.arange(len(self.video_paths))

    def __len__(self):
        return int(np.ceil(len(self.video_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_videos = []
        batch_labels = []

        for i in batch_indices:
            video_dir = self.video_paths[i]
            processed_path = os.path.join(video_dir, 'pose.npy')

            if not os.path.exists(processed_path):
                print(f"Skipping: {processed_path} not found")
                continue

            video_data = np.load(processed_path)

            # Pad/truncate to max_frames
            if video_data.shape[0] < max_frames:
                pad_width = max_frames - video_data.shape[0]
                video_data = np.pad(video_data, ((0, pad_width), (0, 0)), mode='constant')
            else:
                video_data = video_data[:max_frames]

            batch_videos.append(video_data)
            batch_labels.append(self.label_to_index[self.labels[i]])

        if not batch_videos:  # Skip if no valid data
            return np.zeros((0, max_frames, video_data.shape[1])), np.zeros((0,))

        return np.array(batch_videos), np.array(batch_labels)


    def on_epoch_end(self):
        np.random.shuffle(self.indices)

# ============================
# Collect all video paths and labels
# ============================
video_paths = []
labels = []
for sign in tqdm(os.listdir(ASL_DIR), desc="Signs"):
    sign_dir = os.path.join(ASL_DIR, sign)
    if os.path.isdir(sign_dir):
        videos = [v for v in os.listdir(sign_dir) if os.path.isdir(os.path.join(sign_dir, v))]

        # Handle signs with only 1 video
        if len(videos) == 1:
            video_path = os.path.join(sign_dir, videos[0])
            print(f"Augmenting single video for sign: {sign}")
            augmented_dirs = augment_video_pose(video_path, augmentations_needed=10)
            videos += [os.path.basename(d) for d in augmented_dirs]

        # Add all videos (original + augmented) to dataset
        for video in videos:
            video_paths.append(os.path.join(sign_dir, video))
            labels.append(sign)

# Create label mapping
label_to_index = {label: idx for idx, label in enumerate(sorted(set(labels)))}
num_classes = len(label_to_index)

print(f"Total signs: {num_classes}")
print(f"Total video samples: {len(video_paths)}")

# Split data with stratification
train_paths, test_paths, train_labels, test_labels = train_test_split(
    video_paths, labels, test_size=0.2, stratify=labels)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.2, stratify=train_labels)

# Create generators
train_gen = PoseDataGenerator(train_paths, train_labels, label_to_index, set_batch_size)
val_gen = PoseDataGenerator(val_paths, val_labels, label_to_index, set_batch_size)

# ============================
# Model Architecture
# ============================
def build_model(num_classes):
    model = models.Sequential()

    # First LSTM layer needs input shape
    if set_bidirectional:
        model.add(layers.Bidirectional(layers.LSTM(set_lstm_units, return_sequences=True),
                                       input_shape=(max_frames, 132)))
    else:
        model.add(layers.LSTM(set_lstm_units, return_sequences=True, input_shape=(max_frames, 132)))

    for _ in range(set_num_lstm_layers - 1):
        if set_bidirectional:
            model.add(layers.Bidirectional(layers.LSTM(set_lstm_units, return_sequences=True)))
        else:
            model.add(layers.LSTM(set_lstm_units, return_sequences=True))

    model.add(layers.LSTM(set_lstm_units))

    for _ in range(set_num_dense_layers):
        model.add(layers.Dense(set_dense_units, activation=set_activation))
        model.add(layers.Dropout(set_dropout_rate))

    model.add(layers.Dense(num_classes, activation='softmax'))

    optimizer = optimizers.Adam(learning_rate=set_learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


Signs: 100%|██████████| 26/26 [00:00<00:00, 67.53it/s]


Calculated maximum frames across all videos: 54


Signs: 100%|██████████| 26/26 [00:00<00:00, 407.55it/s]

Total signs: 26
Total video samples: 286


In [ ]:
# Loop through the dataset structure
with Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    for word in sorted(os.listdir(ASL_DIR)):
        word_path = os.path.join(ASL_DIR, word)
        if not os.path.isdir(word_path):
            continue

        print(f"Processing: {word}")
        for sequence_folder in tqdm(sorted(os.listdir(word_path))):
            seq_path = os.path.join(word_path, sequence_folder)
            if not os.path.isdir(seq_path):
                continue

            # Skip if pose.npy already exists
            out_path = os.path.join(seq_path, 'pose.npy')
            if os.path.exists(out_path):
                continue

            frames = sorted([f for f in os.listdir(seq_path) if f.endswith('.jpg')])
            if not frames:
                print(f"Warning: No frames in {seq_path}")
                continue

            pose_sequence = []
            for frame_name in frames:
                frame_path = os.path.join(seq_path, frame_name)
                image = cv2.imread(frame_path)
                if image is None:
                    continue
                keypoints = extract_pose_keypoints(image, pose)
                pose_sequence.append(keypoints)

            pose_sequence = np.array(pose_sequence)
            np.save(out_path, pose_sequence)

# ============================
# Callbacks & Training
# ============================
callbacks = [
    callbacks.EarlyStopping(monitor='val_loss', patience=set_patience,
                          min_delta=set_patience_sensitivity, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=set_factor,
                              patience=set_patience//2, min_lr=set_min_lr),
    callbacks.ModelCheckpoint(os.path.join(MODEL_DIR, 'best_model.keras'),
                            save_best_only=False)
]

model_path = os.path.join(MODEL_DIR, 'final_model.keras')
if os.path.exists(model_path):
    print("Loading existing model...")
    model = tf.keras.models.load_model(model_path)
else:
    print("Building new model...")
    model = build_model(num_classes)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=set_num_epochs,
    callbacks=callbacks
)

# ============================
# Save Final Model
# ============================
model.save(os.path.join(MODEL_DIR, 'final_model.h5', ))

Processing: A


100%|██████████| 11/11 [00:00<00:00, 1537.50it/s]


Processing: B


100%|██████████| 11/11 [00:00<00:00, 1636.48it/s]


Processing: C


...

Processing: Y


100%|██████████| 11/11 [00:00<00:00, 1310.05it/s]


Processing: Z


100%|██████████| 11/11 [00:00<00:00, 1101.37it/s]


Loading existing model...
Epoch 1/200
12/12 [==============================] - 17s 659ms/step - loss: 0.7851 - accuracy: 0.7967 - val_loss: 0.4356 - val_accuracy: 0.9565 - lr: 2.5000e-05
Epoch 2/200
12/12 [==============================] - 4s 311ms/step - loss: 0.8134 - accuracy: 0.7473 - val_loss: 0.4268 - val_accuracy: 0.9565 - lr: 2.5000e-05
Epoch 3/200
12/12 [==============================] - 5s 449ms/step - loss: 0.7268 - accuracy: 0.8022 - val_loss: 0.4252 - val_accuracy: 0.9348 - lr: 2.5000e-05
Epoch 4/200
12/12 [==============================] - 5s 427ms/step - loss: 0.7013 - accuracy: 0.8077 - val_loss: 0.4161 - val_accuracy: 0.9565 - lr: 2.5000e-05
Epoch 5/200
12/12 [==============================] - 5s 427ms/step - loss: 0.6226 - accuracy: 0.8571 - val_loss: 0.4218 - val_accuracy: 0.9565 - lr: 2.5000e-05
Epoch 6/200
12/12 [==============================] - 5s 406ms/step - loss: 0.5986 - accuracy: 0.8626 - val_loss: 0.4221 - val_accuracy: 0.9565 - lr: 2.5000e-05
Epoch 7/200
1

**Below is misc. and should not be run normally**

In [ ]:
# ============================
# Save Model in HDF5 Format (without time_major issue)
# ============================
model_save_path = os.path.join(MODEL_DIR, 'best_model_fixed.h5')

# Remove conflicting time_major attribute before saving
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.LSTM):
        layer.time_major = False  # Explicitly set time_major to False to prevent issues

# Save the model
model.save('best_model_fixed.h5', include_optimizer=False)

print(f"Model successfully saved in HDF5 format: {model_save_path}")

Model successfully saved in HDF5 format: /content/drive/MyDrive/ASL_Project/ASL_Model2/best_model_fixed.h5


In [ ]:
# ================
# Data Augmentation
# ================
def augment_video(video_dir, augmentations_needed=1):
    """Create augmented copies of a video by applying random transformations"""
    frame_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.jpg')])
    if not frame_files:
        return []

    parent_dir = os.path.dirname(video_dir)
    base_name = os.path.basename(video_dir)
    existing_video_dirs = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]

    # Start numbering augmented videos
    augmented_index = 0

    augmented_dirs = []
    for i in range(augmentations_needed):
        new_video_name = f'augmented_{augmented_index}'
        new_video_path = os.path.join(parent_dir, new_video_name)
        os.makedirs(new_video_path, exist_ok=True)

        for frame_file in frame_files:
            img = tf.io.read_file(os.path.join(video_dir, frame_file))
            img = tf.image.decode_jpeg(img, channels=channels)

            # Apply random augmentations
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_hue(img, max_delta=0.1)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)

            # Add slight rotation
            angle = tf.random.uniform([], -0.1, 0.1)
            img = tfa.image.rotate(img, angle)

            frame_num = frame_file.split('_')[1].split('.')[0]
            tf.io.write_file(
                os.path.join(new_video_path, f'frame_{frame_num}.jpg'),
                tf.image.encode_jpeg(tf.cast(img * 255, tf.uint8)))

        augmented_dirs.append(new_video_path)
        augmented_index += 1

    return augmented_dirs

# Collect all video paths and labels
video_paths = []
labels = []
for sign in tqdm(os.listdir(ASL_DIR), desc="Signs"):
    sign_dir = os.path.join(ASL_DIR, sign)
    if os.path.isdir(sign_dir):
        videos = [v for v in os.listdir(sign_dir) if os.path.isdir(os.path.join(sign_dir, v))]

        # Handle signs with only 1 video
        if len(videos) == 1:
            video_path = os.path.join(sign_dir, videos[0])
            print(f"Augmenting single video for sign: {sign}")
            augmented_dirs = augment_video(video_path, augmentations_needed=10)
            videos += [os.path.basename(d) for d in augmented_dirs]

Signs: 100%|██████████| 26/26 [00:00<00:00, 393.00it/s]


In [ ]:
import os

root = '/content/drive/MyDrive/ASL_Project/ASL_Alpha'
for word in os.listdir(root):
    word_path = os.path.join(root, word)
    if not os.path.isdir(word_path): continue
    for video in os.listdir(word_path):
        video_path = os.path.join(word_path, video)
        if not os.path.isdir(video_path): continue
        src = os.path.join(video_path, 'processed.npy')
        dst = os.path.join(video_path, 'processed_poses.npy')
        if os.path.exists(src) and not os.path.exists(dst):
            os.rename(src, dst)

In [ ]:
print(tf.__version__)

2.15.1
